Классификация изображений с помощью сверточной нейросети на PyTorch.

In [1]:
!pip install timm

     |████████████████████████████████| 431 kB 5.4 MB/s 


In [2]:
!pip install torch-lr-finder

In [3]:
import numpy as np
import torch
import os
import cv2
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from google.colab import auth
from googleapiclient.discovery import build
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
import albumentations as A
import timm
from torch_lr_finder import LRFinder

Нейросеть будет тренироваться на датасете животных. Данные хранятся на моём Google Диске в архиве animal.zip. Данный архив имеет 5 папок: chiken - курицы, cow - коровы, horse - лошади, pig - свиньи, sheep - овцы.

In [4]:
def download_data(file_id, file_name):
  import io
  from googleapiclient.http import MediaIoBaseDownload

  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()
    
  downloaded.seek(0)
  with open(file_name, "wb") as f:
    f.write(downloaded.read())
  
auth.authenticate_user()
drive_service = build("drive", "v3")

file_id = "1tXCW3nIjN1OW_z1AnRNmgz6ewzT2N-5y"
file_name = "animal.zip"

download_data(file_id, file_name)
!unzip animal.zip

Archive:  animal.zip
   creating: animal/chicken/
  inflating: animal/chicken/__opt__aboutcom__coeus__resources__content_migration__mnn__images__2020__03__rhode-island-hen-7f9b1b93dba8401999c52f85096fbe6c.jpg  
  inflating: animal/chicken/1920x1152_0xac120003_12931955381643790208.jpeg  
  inflating: animal/chicken/2.jpg    
  inflating: animal/chicken/20184925_176_0_1600_1068_1920x0_80_0_0_f8d5b37be51b0c5d2a04bf844b5a2854.jpg  
  inflating: animal/chicken/300px-chicken.jpg  
  inflating: animal/chicken/34de2eb0-83dd-4ce5-906e-f6169bbcbefe.jpg  
  inflating: animal/chicken/755484022851458.jpg  
  inflating: animal/chicken/b616282d.jpg  
  inflating: animal/chicken/Brown-Layer.jpg  
  inflating: animal/chicken/brown-rooster-white-background-isolated-object-live-chicken-one-closeup-farm-animal-92774881.jpg  
  inflating: animal/chicken/brown-rooster-white-background-isolated-object-live-chicken-one-closeup-farm-animal-92919155.jpg  
  inflating: animal/chicken/chicken_home_image_one-750x5

Далее заводятся списки названий файлов, которым присваивается класс в зависимости от номера папки.

In [5]:
filenames = []
labels = []
path = "animal"
for idx, class_dir in enumerate(os.listdir(path)):
  print(f"Берём файлы из папки \"{class_dir}\" и даём им класс {idx}")

  for file in os.listdir(os.path.join(path, class_dir)):
    if not file.endswith((".jpg", ".jpeg", ".png")):
      continue

    filenames.append(os.path.join(path, class_dir, file))
    labels.append(idx)

Берём файлы из папки "sheep" и даём им класс 0
Берём файлы из папки "pig" и даём им класс 1
Берём файлы из папки "horse" и даём им класс 2
Берём файлы из папки "cow" и даём им класс 3
Берём файлы из папки "chicken" и даём им класс 4


Затем датасет разбивается на тренировочную и тестовую выборки в отношении 70/30.

In [6]:
train_filenames, test_filenames, train_labels, test_labels = train_test_split(filenames, labels, test_size=0.3, random_state=42)

Функция add_pad служит для добавления полей к изображениям, чтобы привести их к квадратному формату.

In [7]:
def add_pad(img, shape):
  color_pick = img[0][0]
  padded_img = color_pick * np.ones(shape + img.shape[2:3], dtype=np.uint8)
  x_offset = int((padded_img.shape[0] - img.shape[0]) / 2)
  y_offset = int((padded_img.shape[1] - img.shape[1]) / 2)
  padded_img[x_offset:x_offset + img.shape[0], y_offset:y_offset + img.shape[1]] = img
  return padded_img

Функция resize меняет разрешение изображения.

In [8]:
def resize(img, shape):
  scale = min(shape[0] * 1.0 / img.shape[0], shape[1] * 1.0 / img.shape[1])
  if scale != 1:
    img = cv2.resize(img, dsize=None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
  return img

Функция transform_album применяет аугментации к изображению.

In [9]:
def transform_album(image):
  transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.Transpose(),
    A.OneOf([
      A.IAAAdditiveGaussianNoise(),
      A.GaussNoise(),
    ], p=0.2),
    A.OneOf([
      A.MotionBlur(p=.2),
      A.MedianBlur(blur_limit=3, p=0.1),
      A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
    A.OneOf([
      A.OpticalDistortion(p=0.3),
      A.GridDistortion(p=.1),
      A.IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    A.OneOf([
      A.CLAHE(clip_limit=2),
      A.IAASharpen(),
      A.IAAEmboss(),
      A.RandomBrightnessContrast(),
    ], p=0.3),
    A.HueSaturationValue(p=0.3),
  ])
  random.seed(42)
  augmented_image = transform(image=image)["image"]
  return augmented_image

Класс TrainAnimalDataset возвращает преобразованное изображение по индексу для тренировочной выборки с аугментациями.

In [10]:
class TrainAnimalDataset(Dataset):
  def __init__(self, filenames, labels):
    self._filenames = filenames
    self._labels = labels

  def __len__(self):
    return len(self._filenames)

  def __getitem__(self, idx):
    filename = self._filenames[idx]
    label = self._labels[idx]
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = resize(img, (224, 224))
    img = add_pad(img, (224, 224))
    img = transform_album(img)
    img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1) / 255.
    return img, label

Класс TestAnimalDataset возвращает преобразованное изображение по индексу для тестовой выборки без аугментаций.

In [11]:
class TestAnimalDataset(Dataset):
  def __init__(self, filenames, labels):
    self._filenames = filenames
    self._labels = labels

  def __len__(self):
    return len(self._filenames)

  def __getitem__(self, idx):
    filename = self._filenames[idx]
    label = self._labels[idx]
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = resize(img, (224, 224))
    img = add_pad(img, (224, 224))
    img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1) / 255.
    return img, label

Для тренировочной и тестовой выборки создаются датасеты и даталоадеры. Dataloader просит возвращать Dataset данные по idx и составляет из них батчи.

In [12]:
train_dataset = TrainAnimalDataset(train_filenames, train_labels)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=0)
test_dataset = TestAnimalDataset(test_filenames, test_labels)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=108, num_workers=0)

В качестве исходной модели выступает предобученная SEResNext50. В ней последний слой заменяется на голову, классифицирующую изображение на 1 из 5 классов. CrossEntropyLoss берётся в качестве лосса, а Adam - в качестве оптимизатора. Все слои сети, кроме последнего, замораживаются. Далее он будет обучаться. С помощью библиотеки lr_finder был подобран начальный коэффициент скорости обучения.

In [13]:
model = timm.create_model("seresnext50_32x4d", pretrained=True)
for param in model.parameters():
  param.requires_grad = False

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
optimizer_div10 = torch.optim.Adam(model.parameters(), lr=0.00001)
# lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
# lr_finder.range_test(train_dataloader, end_lr=100, num_iter=100)
# lr_finder.plot()
# lr_finder.reset()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnext50_32x4d_racm-a304a460.pth" to /root/.cache/torch/hub/checkpoints/seresnext50_32x4d_racm-a304a460.pth


Последний слой на 1000 классов удаляется и заменяется на новый с выходом на 5 классов.

In [14]:
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 5)
model.to("cuda")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): SEModule(
        (fc1): Conv2

Функция run_test_on_epoch считает точность модели - на вход передается сама модель, номер эпохи и тестовый лоадер.

In [15]:
def run_test_on_epoch(model, epoch, test_loader):
  model.eval()
  with torch.no_grad():
    test_accuracy = []
    test_real = []
    for batch_x, batch_y in tqdm(test_loader):
      outputs = model(batch_x.to("cuda")).detach().cpu().numpy()
      test_accuracy.append(outputs)
      test_real.append(batch_y.detach().cpu().numpy())
    print("Точность теста эпохи", epoch, "равна", accuracy_score(np.hstack(test_real), np.argmax(np.hstack(test_accuracy), axis=1)))
  model.train()

Обучение модели проходит в течение 25 эпох. Для последних 5 эпох коэффициент скорости обучения уменьшен в 10 раз. Последние 3 слоя размораживаются. В конце каждой эпохи вызывается run_test_on_epoch, чтобы следить за точностью теста эпохи.

In [16]:
ct = 0
for child in model.children():
  ct += 1
  if ct < 47:
    for param in child.parameters():
      param.requires_grad = True

for epoch in tqdm(range(25)):
  for batch in train_dataloader:
    if epoch < 20:
      optimizer.zero_grad()
    else:
      optimizer_div10.zero_grad()
    image, label = batch
    image = image.to("cuda")
    label = label.to("cuda")
    label_pred = model(image)
    loss = criterion(label_pred, label)
    loss.backward()
    if epoch < 20:
      optimizer.step()
    else:
      optimizer_div10.step()

  run_test_on_epoch(model, epoch, test_dataloader)

  4%|▍         | 1/25 [00:08<03:26,  8.60s/it]

Точность теста эпохи 0 равна 0.24444444444444444



  8%|▊         | 2/25 [00:16<03:13,  8.41s/it]

Точность теста эпохи 1 равна 0.4222222222222222



 12%|█▏        | 3/25 [00:25<03:03,  8.35s/it]

...

Точность теста эпохи 22 равна 0.8444444444444444



 96%|█████████▌| 24/25 [03:18<00:08,  8.30s/it]

Точность теста эпохи 23 равна 0.8222222222222222



100%|██████████| 25/25 [03:27<00:00,  8.29s/it]

Точность теста эпохи 24 равна 0.8
